#  3. Transactions  

In the previous session we generated private keys and addresses on the blockchain.  
In this session we will transfer Ethers from your account to another account.    
There are several strategies and tools to do this:  
> Use myetherwallet.com.    
> Use a wallet recommended by the ethereum foundation https://ethereum.org/en/wallets/find-wallet/  
> Generate a unsigned transaction and use a hardware wallet to sign it.  
> Generate a unsigned transaction and use a Hardware Security Module ("HSM") to sign it. Main vendors are listed here: https://webencrypt.org/hsm/  
>
All the strategies listed above perform three logical steps:  

**Step 1: Create a transaction based on the following information** 
* account (or accounts) which the Ethers are to be sent from (**sender account**)    
* account (or accounts) which the Ethers are to be sent to (**recipient account**)   
* the transaction fees that you intend to offer[1] to miners to prioritise your transaction over those submitted by other users of the blockchain.  
[1] Check the current market rates for fees here https://beaconcha.in/gasnow for the main network or here https://sepolia.beaconcha.in/gasnow for the test network. Of course, you can offer less than the lowest price observed but your transaction would be at risk of getting ignored by miners.

**Step 2: Sign the transaction**  

💭 To be valid, the transaction has to be signed with the private key of the **sender account**.  

**Step 3: Post the transaction to the blockchain**  

The transaction is posted on your node and from there broadcast on the blockchain through the other nodes (peers) connected to your node.  
If you do not have a node, you can use a node that is made available by node-as-a-service vendors (like Infura or  Alchemy).  
Alternatively, the transaction can be posted to free online services like https://sepolia.beaconcha.in/tools/broadcast or - only for mainnet transactions - https://live.blockcypher.com/eth/pushtx/?t=None  

💡 A signed transaction does not contain confidential information and cannot be tampered with.  

Once posted, signed transactions are available to miners, who can include them in a block that they intend to mine. Once mined the block is appended to the blockchain and is considered final.  

Inadequate processes and system controls around the three steps above, may result in a loss of assets and in the failure to transfer assets on a timely basis.  
Specifically:
* Issues affecting step one may result in erroneous or fraudulent transactions that - if unchecked prior to signing - may move assets to inappropriate accounts.  
* Inadequate fees offered in step one may also result in transactions being left behind by miners, remaining pending in the mempool, and eventually dropped.  
* Step two is critical, as private keys are retrieved from the secure storage and used to sign transactions.  
* Issues affecting step three may only affect the timeliness of transactions.

The code below defines the functions to generate an Ethereum account from a given private key (similar to those that you have already seen in the previous module) and post a transaction on the Ethereum blockchain.    

#Cell 1

In [ ]:
print("Generating functions...") 

%pip install web3 -q
%pip install eth_account -q

from eth_account import Account
from web3 import Web3, HTTPProvider
from web3.gas_strategies.rpc import rpc_gas_price_strategy
import binascii


def pad64(s):
    s = str(hex(s))[2:]
    return s.zfill(64)
def eth_account(private_key):
    return Account.from_key(pad64(private_key)).address, pad64(private_key)

def str2hex(s):
    h = ""
    for c in s:
        h += hex(ord(c))[2:]
    return int(h, 16) , len(h)  

def balance(account):
    w3 = Web3(HTTPProvider('https://endpoints.omniatech.io/v1/eth/sepolia/public'))
    return  w3.from_wei(w3.eth.get_balance(account),'ether')

def create_transaction(account_from, address_to, value, gas_price):
#    w3 = Web3(HTTPProvider('https://sepolia.infura.io/v3/fae476cc30ff43e0813948aca32f409d'))
    w3 = Web3(HTTPProvider('https://endpoints.omniatech.io/v1/eth/sepolia/public'))
    return dict (nonce=w3.eth.get_transaction_count(account_from),
            #maxFeePerGas = w3.to_wei(gas_price, 'gwei'),
            gas = 21000,
            gasPrice = w3.to_wei(gas_price, 'gwei'),
            to = address_to,
            chainId = 11155111,
            value = w3.to_wei(value, "ether"))

def sign_transaction(transaction, private_key):
    w3 = Web3(HTTPProvider('https://sepolia.infura.io/v3/fae476cc30ff43e0813948aca32f409d'))
    signed = w3.eth.account.sign_transaction(transaction, private_key)
    return signed.rawTransaction

def post_transaction(signed_transaction):
    w3 = Web3(HTTPProvider('https://sepolia.infura.io/v3/fae476cc30ff43e0813948aca32f409d'))
    tx_hash=w3.eth.send_raw_transaction(signed_transaction)
    return tx_hash

print("Functions generated, you can now use the functions on this workbook to derive Ethereum addresses from a given private key and transfer funds.")

#Cell 2

**Create an account and request test Ethers**  
Use the functions defined above to generate an account and request test funds from your instructor.  
We will use Sepolia a public Ethereum test network.  
Check that you have received some tokens using https://sepolia.beaconcha.in/ (Sepolia block explorer).  
Remember to save your public key and private keys somewhere where you can retrieve them (use notepad and store them in variables on your workbook).  
Use variables to get the values returned from the functions.  You can use the funciton print(*variable*) to dispaly the content of the variable. 

In [ ]:
#Your code


#Cell 3

**Group Exercise**  
Work in pairs to transfer two million wei (10^-12 ethers, i.e. 0.000000000002 Ethers) to each other.  
Remember to offer a good gas price to ensure that your transaction is executed as quickly as possible.    
💡 You can check the current rates of gas on Sepolia here https://sepolia.beaconcha.in/gasnow  
Check that the transaction has gone through using the Sepolia block explorer.  



#Cell 4

In [ ]:
#Your code

#Cell 5

**Cancel a Transaction**  

Until transacitons have been picked by miners, added to a new block and the block is mined, transaction are stored in a shared queue called TX-QUEUE or **mempool** (using the Bitcoin terminology). When in the mempool, transactions are **pending** (if the sequence is consistent) or **queued** (if the sequence is not consistent). 
💭 It is possible to cancel a pending transaction by posting a **new** transaction that: 
* sends 0 ethers to the same account which the original transaction intended to transfer ethers from  
* has higher gas fees  
* has the same nonce as the pending transaction.

The functions defined for you will replace a transaction while this is still in the mempool as long as the same amount is sent.  

**Exercise**  
Submit a transaction with a low gas price and check its status on the sepolia explorer.  
Whilst the transaction is stuck in the mempool, replace it with another one with higher fees.  

#Cell 6

In [ ]:
#Your code

#Cell 7